# Midterm Project: Machine Learning Modeling
## 一、数据加载与准备

In [84]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet , SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import re
import warnings

# 加载数据
train_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_train.csv")
test_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_test.csv")
detail_df = pd.read_csv("C:/Users/86158/Desktop/Mid_Test/Data/ruc_Class25Q1_details.csv")

warnings.filterwarnings('ignore') 

## 二、数据预处理
### 2.1 数据检查与合并

In [85]:
print("训练数据列名:", train_df.columns.tolist())
print("详情数据列名:", detail_df.columns.tolist())
print("详预测数据列名:", test_df.columns.tolist())

训练数据列名: ['城市', '区域', '板块', '环线', '小区名称', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份']
详情数据列名: ['区县', '名称', '城市', '板块', '环线位置', '小区地址', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y']
详预测数据列名: ['ID', '城市', '区域', '板块', '环线', '小区名称', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易时间', '交易权属', '上次交易', '房屋用途', '房屋年限', '产权所属', '抵押信息', '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '年份']


### 2.2 预处理函数构建（特征工程）

In [ ]:
def improved_preprocessing(df, is_train=True, target_enc=None):
    # 记录初始行数
    initial_rows = len(df)
    print(f"\n{'训练数据' if is_train else '测试数据'}初始行数: {initial_rows}")
    
    processed_df = df.copy()
    
    # 1. 智能数据合并
    if '小区名称' in processed_df.columns and '名称' in detail_df.columns:
        print("\n正在合并小区详情数据...")
        detail_df_clean = detail_df.drop_duplicates(subset=['名称'])
        processed_df = pd.merge(
            processed_df, 
            detail_df_clean, 
            left_on='小区名称', 
            right_on='名称', 
            how='left'
        )
        processed_df['有小区详情'] = processed_df['名称'].notna().astype(int)
        processed_df = processed_df.drop('名称', axis=1, errors='ignore')
    
    # 2. 删除无用特征及泄密信息
    drop_cols = [
        '房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行',
        'lon', 'lat', '小区名称', '房屋年限', '上次交易', '年份'
    ]
    drop_cols = [col for col in drop_cols if col in processed_df.columns]
    print(f"\n将删除以下列: {drop_cols}")
    processed_df = processed_df.drop(drop_cols, axis=1)
    
    # 3. 高级时间特征工程
    if '交易时间' in processed_df.columns:
        try:
            processed_df['交易时间'] = pd.to_datetime(processed_df['交易时间'], errors='coerce')
            processed_df['交易年份'] = processed_df['交易时间'].dt.year
            processed_df['交易月份'] = processed_df['交易时间'].dt.month
            processed_df['交易季度'] = processed_df['交易时间'].dt.quarter
            processed_df['是否年底'] = processed_df['交易时间'].dt.month.isin([11, 12]).astype(int)
            processed_df = processed_df.drop('交易时间', axis=1)
            print("\n已处理日期特征")
        except Exception as e:
            print("日期处理错误:", e)
    
    # 4. 增强型数值特征处理 - 修正建筑面积处理
    if '建筑面积' in processed_df.columns:
        try:
            # 更健壮的面积处理方式
            processed_df['建筑面积'] = (
                processed_df['建筑面积']
                .astype(str)
                .str.replace('[^\d.]', '', regex=True) 
                .replace('', np.nan)  
                .astype(float)
            )
            processed_df['建筑面积_ori'] = processed_df['建筑面积']
            
            # 创建面积分段特征
            bins = [0, 60, 90, 120, 150, 200, 300, float('inf')]
            labels = ['微型', '小型', '中小型', '中型', '中大', '大型', '超大型']
            processed_df['面积等级'] = pd.cut(
                processed_df['建筑面积'], 
                bins=bins, 
                labels=labels,
                right=False
            )
            print("\n已处理建筑面积特征")
        except Exception as e:
            print("建筑面积处理错误:", e)
            processed_df['建筑面积'] = pd.to_numeric(processed_df['建筑面积'], errors='coerce')
            processed_df['建筑面积_ori'] = processed_df['建筑面积']
    
    # 5. 增强型分类特征处理
    # 建筑结构增强处理
    if '建筑结构' in processed_df.columns:
        structure_mapping = {
            '钢混结构': '现代',
            '钢结构': '现代',
            '框架结构': '现代',
            '混合结构': '传统',
            '砖混结构': '传统',
            '砖木结构': '传统'
        }
        processed_df['建筑结构类型'] = processed_df['建筑结构'].map(structure_mapping)
        processed_df['建筑结构_现代'] = processed_df['建筑结构类型'].apply(lambda x: 1 if x == '现代' else 0)
    
    # 装修情况增强处理
    if '装修情况' in processed_df.columns:
        processed_df['装修等级'] = processed_df['装修情况'].map({
            '精装': 3,
            '简装': 2,
            '毛坯': 1,
            '其他': 0
        }).fillna(0)
    
    # 6. 创新性特征工程
    # 从房屋户型提取房间数
    if '房屋户型' in processed_df.columns:
        processed_df['室数'] = processed_df['房屋户型'].str.extract(r'(\d+)室')[0].astype(float)
        processed_df['厅数'] = processed_df['房屋户型'].str.extract(r'(\d+)厅')[0].astype(float)
        processed_df['卫数'] = processed_df['房屋户型'].str.extract(r'(\d+)卫')[0].astype(float)
        processed_df['厨数'] = processed_df['房屋户型'].str.extract(r'(\d+)厨')[0].astype(float)
        
        # 创建房间比例特征
        processed_df['厅室比'] = processed_df['厅数'] / (processed_df['室数'] + 1e-6)
        processed_df['卫室比'] = processed_df['卫数'] / (processed_df['室数'] + 1e-6)
    
    # 7. 高级空间特征
    if 'coord_x' in processed_df.columns and 'coord_y' in processed_df.columns:
        # 计算到城市中心的相对距离（假设中心点为数据中位数）
        center_x, center_y = processed_df['coord_x'].median(), processed_df['coord_y'].median()
        processed_df['距市中心距离'] = np.sqrt(
            (processed_df['coord_x'] - center_x)**2 + 
            (processed_df['coord_y'] - center_y)**2
        )
    
    # 8. 小区级别特征聚合
    if '小区地址' in processed_df.columns:
        # 小区平均价格特征（仅训练集）
        if is_train and '价格' in processed_df.columns:
            community_stats = processed_df.groupby('小区地址')['价格'].agg(['mean', 'median', 'std']).add_prefix('小区价格_')
            processed_df = processed_df.merge(community_stats, on='小区地址', how='left')
    
    # 修改第9部分（交互特征增强）的代码：
    if '建筑面积' in processed_df.columns and '室数' in processed_df.columns:
        processed_df['人均面积'] = processed_df['建筑面积'] / (processed_df['室数'] + 1e-6)
    # 只在训练集计算面积价格比
    if is_train and '价格' in processed_df.columns:
        processed_df['面积价格比'] = processed_df['价格'] / (processed_df['建筑面积'] + 1e-6)
    
    # 10. 高级编码技术
    high_card_cols = ['区域', '板块', '开发商', '物业公司']
    high_card_cols = [col for col in high_card_cols if col in processed_df.columns]
    
    if high_card_cols:
        if is_train and '价格' in processed_df.columns:
            # 使用目标编码与频率编码结合
            processed_df['log_price'] = np.log1p(processed_df['价格'])
            for col in high_card_cols:
                # 目标编码
                target_mean = processed_df.groupby(col)['log_price'].mean().to_dict()
                processed_df[f'{col}_target_enc'] = processed_df[col].map(target_mean)
                
                # 频率编码
                freq = processed_df[col].value_counts(normalize=True).to_dict()
                processed_df[f'{col}_freq_enc'] = processed_df[col].map(freq)
                
                # 保存编码器状态
                if target_enc is None:
                    target_enc = {}
                target_enc[col] = {'target': target_mean, 'freq': freq}
                
            processed_df = processed_df.drop(high_card_cols, axis=1)

        elif not is_train and target_enc is not None:
            # 测试集应用编码
            for col in high_card_cols:
                if col in target_enc:
                    processed_df[f'{col}_target_enc'] = processed_df[col].map(target_enc[col]['target'])
                    processed_df[f'{col}_freq_enc'] = processed_df[col].map(target_enc[col]['freq'])
            processed_df = processed_df.drop(high_card_cols, axis=1)
    
    # 最终检查
    final_rows = len(processed_df)
    print(f"\n预处理完成，最终行数: {final_rows}")
    if initial_rows != final_rows:
        print(f"警告: 行数发生变化 (初始: {initial_rows}, 最终: {final_rows})")
    else:
        print("行数保持一致")
    
    return processed_df, target_enc

### 2.3 执行数据预处理

In [95]:
# 重新运行预处理
try:
    print("\n=== 预处理训练数据 ===")
    train_processed, target_encoder = improved_preprocessing(train_df, is_train=True)
    
    print("\n=== 预处理预测数据 ===")
    test_processed, _ = improved_preprocessing(test_df, is_train=False, target_enc=target_encoder)
    
    # 检查结果
    if 'train_processed' in locals() and 'test_processed' in locals():
        print("\n预处理成功完成！")
        print("训练数据形状:", train_processed.shape)
        print("预测数据形状:", test_processed.shape)
        print("\n训练集前几列:", train_processed.columns.tolist()[:10])
        print("预测试集前几列:", test_processed.columns.tolist()[:10])
    else:
        print("\n预处理未能完成")
except Exception as e:
    print("\n执行预处理时出错:", str(e))
    import traceback
    traceback.print_exc()


=== 预处理训练数据 ===

训练数据初始行数: 84133

正在合并小区详情数据...

将删除以下列: ['房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '小区名称', '房屋年限', '上次交易', '年份']

已处理日期特征

已处理建筑面积特征

预处理完成，最终行数: 84133
行数保持一致

=== 预处理预测数据 ===

测试数据初始行数: 14786

正在合并小区详情数据...

将删除以下列: ['房屋优势', '核心卖点', '户型介绍', '周边配套', '交通出行', 'lon', 'lat', '小区名称', '房屋年限', '上次交易', '年份']

已处理日期特征

已处理建筑面积特征

预处理完成，最终行数: 14786
行数保持一致

预处理成功完成！
训练数据形状: (84133, 69)
预测数据形状: (14786, 64)

训练集前几列: ['城市_x', '板块_x', '环线', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构_x']
预测试集前几列: ['ID', '城市_x', '板块_x', '环线', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向', '建筑结构_x']


### 2.4 异常值处理

In [97]:
print(train_processed.columns)

y = train_processed['log_price']
X = train_processed.drop(['价格', 'log_price', '建筑面积_ori'], axis=1, errors='ignore')

# 异常值处理
print("\n=== 异常值处理 ===")
numeric_cols = X.select_dtypes(include=np.number).columns

for col in numeric_cols:
    if X[col].nunique() > 10:  # 只对连续型数值特征处理
        lower = X[col].quantile(0.01)
        upper = X[col].quantile(0.99)
        print(f"{col}: 截断范围 [{lower:.2f}, {upper:.2f}]")
            
        # 保存原始值计数用于报告
        original_count = len(X)
        outliers_low = (X[col] < lower).sum()
        outliers_high = (X[col] > upper).sum()
            
        # 执行截断
        X[col] = X[col].clip(lower, upper)
            
        # 打印处理结果
        print(f"  处理了 {outliers_low + outliers_high} 个异常值 "
                  f"(低于1%: {outliers_low}, 高于99%: {outliers_high})")


Index(['城市_x', '板块_x', '环线', '价格', '房屋户型', '所在楼层', '建筑面积', '套内面积', '房屋朝向',
       '建筑结构_x', '装修情况', '梯户比例', '配备电梯', '别墅类型', '交易权属', '房屋用途', '产权所属',
       '抵押信息', '区县', '城市_y', '板块_y', '环线位置', '小区地址', '物业类别', '建筑年代', '房屋总数',
       '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '建筑结构_y', '物业办公电话', '产权描述', '供水',
       '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '有小区详情',
       '交易年份', '交易月份', '交易季度', '是否年底', '建筑面积_ori', '面积等级', '装修等级', '室数', '厅数',
       '卫数', '厨数', '厅室比', '卫室比', '距市中心距离', '小区价格_mean', '小区价格_median',
       '小区价格_std', '人均面积', '面积价格比', 'log_price', '区域_target_enc',
       '区域_freq_enc', '开发商_target_enc', '开发商_freq_enc', '物业公司_target_enc',
       '物业公司_freq_enc'],
      dtype='object')

=== 异常值处理 ===
板块_x: 截断范围 [27.00, 807.00]
  处理了 1256 个异常值 (低于1%: 775, 高于99%: 481)
建筑面积: 截断范围 [30.18, 279.61]
  处理了 1684 个异常值 (低于1%: 842, 高于99%: 842)
区县: 截断范围 [3.00, 99.97]
  处理了 1427 个异常值 (低于1%: 642, 高于99%: 785)
板块_y: 截断范围 [27.00, 807.00]
  处理了 1197 个异常值 (低于1%: 751, 高于99%: 446)
容 积

### 2.5 缺失值处理

In [98]:
print("\n=== 缺失值处理 ===")
print("处理前缺失值统计:")
print(X.isna().sum().sort_values(ascending=False))

# 数值型缺失值
for col in numeric_cols:
    if X[col].isna().any():
        median_val = X[col].median()
        X[col] = X[col].fillna(median_val if not np.isnan(median_val) else 0)

# 类别型缺失值
categorical_cols = X.select_dtypes(include='object').columns
for col in categorical_cols:
    if X[col].isna().any():
        X[col] = X[col].fillna('missing')

print("\n处理后缺失值统计:")
print(X.isna().sum().sort_values(ascending=False))

y_price = train_processed['价格']  # 用于价格级别评估


=== 缺失值处理 ===
处理前缺失值统计:
抵押信息             84133
别墅类型             83384
套内面积             58987
物业办公电话           56272
环线位置             42758
                 ...  
是否年底                 0
交易月份                 0
区域_freq_enc          0
面积价格比                0
区域_target_enc        0
Length: 66, dtype: int64

处理后缺失值统计:
城市_x               0
板块_x               0
环线                 0
房屋户型               0
所在楼层               0
                  ..
区域_freq_enc        0
开发商_target_enc     0
开发商_freq_enc       0
物业公司_target_enc    0
物业公司_freq_enc      0
Length: 66, dtype: int64


## 三、模型构建与训练

### 3.1 预处理管道构建

In [99]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=111)

numeric_features = X_train.select_dtypes(include=np.number).columns
categorical_features = X_train.select_dtypes(include='object').columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

### 3.2 模型配置

In [112]:
models = {
    'OLS': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression(n_jobs=-1))
    ]),
    'Lasso': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='l1',
            alpha=0.0001,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ]),
    'Ridge': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='l2',
            alpha=0.0001,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ]),
    'ElasticNet': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SGDRegressor(
            penalty='elasticnet',
            alpha=0.0001,
            l1_ratio=0.5,
            max_iter=2000,
            tol=1e-3,
            random_state=111))
    ])
}

## 四、模型评估与结果分析

### 4.1 模型训练与评估

In [124]:
results = {}
for name, pipe in models.items():
    print(f"\n训练 {name}...")
    try:
        pipe.fit(X_train, y_train)
        
        # 常规评估
        y_pred = pipe.predict(X_val)
        price_pred = (np.exp(y_pred) * X_val['建筑面积'])/10000 if '建筑面积' in X_val.columns else np.exp(y_pred)
        price_true = (np.exp(y_val) * X_val['建筑面积'])/10000 if '建筑面积' in X_val.columns else np.exp(y_val)
        
        # 6折交叉验证（在训练集上）
        cv_scores = cross_val_score(
            pipe, X_train, y_train,
            cv=6,
            scoring='neg_mean_absolute_error',
            n_jobs=-1
        )
        
        results[name] = {
            'In-sample MAE': mean_absolute_error(y_train, pipe.predict(X_train)),
            'Out-sample MAE': mean_absolute_error(y_val, y_pred),
            'Price MAE': mean_absolute_error(price_true, price_pred),
            'Price RMSE': np.sqrt(mean_squared_error(price_true, price_pred)),
            'Cross-validation MAE': -cv_scores.mean()  # 交叉验证结果
        }
    except Exception as e:
        print(f"{name}训练失败:", str(e))
        results[name] = {k: np.nan for k in ['In-sample MAE', 'Out-sample MAE', 'Price MAE', 'Price RMSE', 'Cross-validation MAE']}


训练 OLS...

训练 Lasso...

训练 Ridge...

训练 ElasticNet...


### 4.2 结果展示与分析

In [125]:
metrics = pd.DataFrame(results).T

# 确定最佳模型
valid_results = metrics.dropna()
if not valid_results.empty:
    best_model = valid_results['Cross-validation MAE'].idxmin()
    metrics.loc['Best Model'] = metrics.loc[best_model]
    
print("\n=== 测试结果 ===")
print(metrics[['In-sample MAE', 'Out-sample MAE', 'Cross-validation MAE', 'Price MAE', 'Price RMSE']])




=== 测试结果 ===
            In-sample MAE  Out-sample MAE  Cross-validation MAE    Price MAE  \
OLS              0.061584        0.076010              0.077528  2324.816905   
Lasso            0.125739        0.127960              0.129991  4470.281775   
Ridge            0.109788        0.113086              0.116385  3768.794563   
ElasticNet       0.118993        0.121590              0.123954  4244.695146   
Best Model       0.061584        0.076010              0.077528  2324.816905   

              Price RMSE  
OLS         13272.349054  
Lasso       27797.323367  
Ridge       23906.170790  
ElasticNet  26756.957933  
Best Model  13272.349054  


## 五、预测结果生成

### 5.1 测试集预测准备

In [126]:
# 确保测试数据包含所有训练特征
missing_cols = set(X_train.columns) - set(test_processed.columns)
for col in missing_cols:
    test_processed[col] = 0  # 用0填充缺失特征

# 确保列顺序一致
X_test = test_processed[X_train.columns]

# 在预测前添加数据检查
print("\n=== 预测前数据检查 ===")
print("测试集建筑面积统计:")
print(test_processed['建筑面积'].describe())

print("\n模型预测值统计:")
test_log_pred_sample = models['OLS'].predict(X_test.head())
print(pd.Series(test_log_pred_sample).describe())


=== 预测前数据检查 ===
测试集建筑面积统计:
count    14786.000000
mean        92.494791
std         39.914068
min         18.760000
25%         66.780000
50%         87.985000
75%        109.980000
max        594.380000
Name: 建筑面积, dtype: float64

模型预测值统计:
count     5.000000
mean     14.025761
std       0.580067
min      13.024830
25%      14.099258
50%      14.149029
75%      14.420437
max      14.435251
dtype: float64


### 5.2 执行预测并保存结果

In [128]:
# 生成所有模型的预测结果
predictions = {}
# 修改预测部分代码
for model_name, model in models.items():
    try:
        test_log_pred = model.predict(X_test)
        
        # 转换回原始价格（添加异常值处理）
        if '建筑面积' in test_processed.columns:
            # 确保没有无限大或NA值
            test_log_pred = np.clip(test_log_pred, -100, 100)  # 防止指数爆炸
            test_price_pred = np.exp(test_log_pred) * test_processed['建筑面积']
            
            # 处理可能的NA/inf
            test_price_pred = np.nan_to_num(test_price_pred, posinf=1e8, neginf=0)
            test_price_pred = np.clip(test_price_pred, 0, 1e8)  # 限制价格范围
        else:
            test_price_pred = np.exp(np.clip(test_log_pred, -100, 100))
        
        # 保存预测结果（确保为有限值）
        predictions[model_name] = np.round(test_price_pred).astype(int)
        
        # 创建DataFrame并保存
        submission = pd.DataFrame({
            'ID': test_df['ID'],
            'Price': predictions[model_name]
        })
        submission.to_csv(f"prediction_{model_name}.csv", index=False)
        
    except Exception as e:
        print(f"模型 {model_name} 预测失败: {str(e)}")
        predictions[model_name] = None

# 输出预测统计信息
print("\n所有模型预测结果统计:")
for model_name, pred in predictions.items():
    if pred is not None:
        print(f"{model_name}: 预测值范围 [{pred.min():.0f}, {pred.max():.0f}]")
    else:
        print(f"{model_name}: 无有效预测结果")



所有模型预测结果统计:
OLS: 预测值范围 [2134032, 100000000]
Lasso: 预测值范围 [3528360, 100000000]
Ridge: 预测值范围 [3302893, 100000000]
ElasticNet: 预测值范围 [3432063, 100000000]


### 5.3 最终结果

In [ ]:
metrics = pd.DataFrame(results).T
metrics['Datahub Score'] = [78.422, 73.782, 74.505, 74.798]  # 按OLS,Lasso,Ridge,ElasticNet顺序

print("\n=== 最终结果 ===")
# 确定最佳模型（基于Cross-validation MAE）
valid_results = metrics.dropna()
if not valid_results.empty:
    best_model = valid_results['Cross-validation MAE'].idxmin()
    metrics.loc['Best Model'] = metrics.loc[best_model]
    metrics.at['Best Model', 'Datahub Score'] = metrics.loc[best_model, 'Datahub Score']

# 输出表格（保留3位小数）
final_table = metrics[['In-sample MAE', 'Out-sample MAE', 'Cross-validation MAE', 'Datahub Score']]
final_table.columns = ['In sample', 'Out of sample', 'Cross-validation', 'Datahub Score']
print("\n| Metrics        | In sample | Out of sample | Cross-validation | Datahub Score |")
print("|----------------|-----------|---------------|-------------------|---------------|")
for model in final_table.index:
    print(f"| {model:14} | {final_table.loc[model, 'In sample']:>9.3f} | "
          f"{final_table.loc[model, 'Out of sample']:>13.3f} | "
          f"{final_table.loc[model, 'Cross-validation']:>17.3f} | "
          f"{final_table.loc[model, 'Datahub Score']:>13.3f} |")